# Ungraded Lab: Build a Multi-output Model

In this lab, we'll show how you can build models with more than one output. The dataset we will be working on is available from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Energy+efficiency). It is an Energy Efficiency dataset which uses the bulding features (e.g. wall area, roof area) as inputs and has two outputs: Cooling Load and Heating Load. Let's see how we can build a model to train on this data.

## Imports

In [1]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input
from sklearn.model_selection import train_test_split

## Utilities

We define a few utilities for data conversion and visualization to make our code more neat.

In [2]:
def format_output(data):
    y1 = data.pop('Y1')
    y1 = np.array(y1)
    y2 = data.pop('Y2')
    y2 = np.array(y2)
    return y1, y2


def norm(x):
    return (x - train_stats['mean']) / train_stats['std']


def plot_diff(y_true, y_pred, title=''):
    plt.scatter(y_true, y_pred)
    plt.title(title)
    plt.xlabel('True Values')
    plt.ylabel('Predictions')
    plt.axis('equal')
    plt.axis('square')
    plt.xlim(plt.xlim())
    plt.ylim(plt.ylim())
    plt.plot([-100, 100], [-100, 100])
    plt.show()


def plot_metrics(metric_name, title, ylim=5):
    plt.title(title)
    plt.ylim(0, ylim)
    plt.plot(history.history[metric_name], color='blue', label=metric_name)
    plt.plot(history.history['val_' + metric_name], color='green', label='val_' + metric_name)
    plt.show()

## Prepare the Data

We download the dataset and format it for training.

In [7]:
# Get the data from UCI dataset
URL = 'https://github.com/lucasdavid/manifold-learning/blob/master/datasets/energy/ENB2012_data.xlsx?raw=true'

# Use pandas excel reader
df = pd.read_excel(URL)
df = df.sample(frac=1).reset_index(drop=True)

# Split the data into train and test with 80 train / 20 test
train, test = train_test_split(df, test_size=0.2)
train_stats = train.describe()

# Get Y1 and Y2 as the 2 outputs and format them as np arrays
train_stats.pop('Y1')
train_stats.pop('Y2')
train_stats = train_stats.transpose()
train_Y = format_output(train)
test_Y = format_output(test)

# Normalize the training and test data
norm_train_X = norm(train)
norm_test_X = norm(test)

## Build the Model

Here is how we'll build the model using the functional syntax. Notice that we can specify a list of outputs (i.e. `[y1_output, y2_output]`) when we instantiate the `Model()` class.

In [8]:
# Define model layers.
input_layer = Input(shape=(len(train .columns),))
first_dense = Dense(units='128', activation='relu')(input_layer)
second_dense = Dense(units='128', activation='relu')(first_dense)

# Y1 output will be fed directly from the second dense
y1_output = Dense(units='1', name='y1_output')(second_dense)
third_dense = Dense(units='64', activation='relu')(second_dense)

# Y2 output will come via the third dense
y2_output = Dense(units='1', name='y2_output')(third_dense)

# Define the model with the input layer and a list of output layers
model = Model(inputs=input_layer, outputs=[y1_output, y2_output])

print(model.summary())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 8)]          0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 128)          1152        input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 128)          16512       dense[0][0]                      
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 64)           8256        dense_1[0][0]                    
______________________________________________________________________________________________

## Configure parameters

We specify the optimizer as well as the loss and metrics for each output.

In [9]:
# Specify the optimizer, and compile the model with loss functions for both outputs
optimizer = tf.keras.optimizers.SGD(lr=0.001)
model.compile(optimizer=optimizer,
              loss={'y1_output': 'mse', 'y2_output': 'mse'},
              metrics={'y1_output': tf.keras.metrics.RootMeanSquaredError(),
                       'y2_output': tf.keras.metrics.RootMeanSquaredError()})

## Train the Model

In [ ]:
# Train the model for 500 epochs
history = model.fit(norm_train_X, train_Y,
                    epochs=500, batch_size=10, validation_data=(norm_test_X, test_Y))

Train on 614 samples, validate on 154 samples
Epoch 1/500
614/614 [==============================] - 1s 992us/sample - loss: 231.2171 - y1_output_loss: 104.5817 - y2_output_loss: 124.6851 - y1_output_root_mean_squared_error: 10.2746 - y2_output_root_mean_squared_error: 11.2094 - val_loss: 47.8887 - val_y1_output_loss: 19.5126 - val_y2_output_loss: 29.3318 - val_y1_output_root_mean_squared_error: 4.3514 - val_y2_output_root_mean_squared_error: 5.3809
Epoch 2/500
614/614 [==============================] - 0s 139us/sample - loss: 32.9020 - y1_output_loss: 12.4073 - y2_output_loss: 20.2811 - y1_output_root_mean_squared_error: 3.5357 - y2_output_root_mean_squared_error: 4.5168 - val_loss: 31.3263 - val_y1_output_loss: 13.1619 - val_y2_output_loss: 18.2813 - val_y1_output_root_mean_squared_error: 3.5738 - val_y2_output_root_mean_squared_error: 4.3075
Epoch 3/500
614/614 [==============================] - 0s 141us/sample - loss: 25.1550 - y1_output_loss: 10.2019 - y2_output_loss: 14.9041 - y1

614/614 [==============================] - 0s 130us/sample - loss: 16.9005 - y1_output_loss: 5.9508 - y2_output_loss: 10.9735 - y1_output_root_mean_squared_error: 2.4252 - y2_output_root_mean_squared_error: 3.3195 - val_loss: 14.7530 - val_y1_output_loss: 6.1993 - val_y2_output_loss: 8.6016 - val_y1_output_root_mean_squared_error: 2.4612 - val_y2_output_root_mean_squared_error: 2.9488
Epoch 22/500
614/614 [==============================] - 0s 132us/sample - loss: 14.1997 - y1_output_loss: 5.7629 - y2_output_loss: 8.6557 - y1_output_root_mean_squared_error: 2.3691 - y2_output_root_mean_squared_error: 2.9304 - val_loss: 26.5154 - val_y1_output_loss: 10.9034 - val_y2_output_loss: 16.0881 - val_y1_output_root_mean_squared_error: 3.2287 - val_y2_output_root_mean_squared_error: 4.0113
Epoch 23/500
614/614 [==============================] - 0s 138us/sample - loss: 13.4349 - y1_output_loss: 5.0348 - y2_output_loss: 8.3836 - y1_output_root_mean_squared_error: 2.2443 - y2_output_root_mean_square

614/614 [==============================] - 0s 127us/sample - loss: 8.9722 - y1_output_loss: 2.7892 - y2_output_loss: 6.2553 - y1_output_root_mean_squared_error: 1.6496 - y2_output_root_mean_squared_error: 2.5002 - val_loss: 17.1729 - val_y1_output_loss: 4.8928 - val_y2_output_loss: 11.9874 - val_y1_output_root_mean_squared_error: 2.2223 - val_y2_output_root_mean_squared_error: 3.4978
Epoch 42/500
614/614 [==============================] - 0s 127us/sample - loss: 8.4669 - y1_output_loss: 2.7999 - y2_output_loss: 5.7868 - y1_output_root_mean_squared_error: 1.6693 - y2_output_root_mean_squared_error: 2.3834 - val_loss: 13.1526 - val_y1_output_loss: 2.9847 - val_y2_output_loss: 10.4731 - val_y1_output_root_mean_squared_error: 1.7407 - val_y2_output_root_mean_squared_error: 3.1816
Epoch 43/500
614/614 [==============================] - 0s 128us/sample - loss: 8.9623 - y1_output_loss: 2.8135 - y2_output_loss: 6.2131 - y1_output_root_mean_squared_error: 1.6798 - y2_output_root_mean_squared_er

Epoch 62/500
614/614 [==============================] - 0s 126us/sample - loss: 4.4723 - y1_output_loss: 1.2681 - y2_output_loss: 3.1750 - y1_output_root_mean_squared_error: 1.1293 - y2_output_root_mean_squared_error: 1.7880 - val_loss: 4.3410 - val_y1_output_loss: 1.3379 - val_y2_output_loss: 2.9468 - val_y1_output_root_mean_squared_error: 1.1542 - val_y2_output_root_mean_squared_error: 1.7346
Epoch 63/500
614/614 [==============================] - 0s 130us/sample - loss: 4.0611 - y1_output_loss: 1.0280 - y2_output_loss: 3.0467 - y1_output_root_mean_squared_error: 1.0123 - y2_output_root_mean_squared_error: 1.7425 - val_loss: 7.9371 - val_y1_output_loss: 1.5377 - val_y2_output_loss: 6.2692 - val_y1_output_root_mean_squared_error: 1.2412 - val_y2_output_root_mean_squared_error: 2.5292
Epoch 64/500
614/614 [==============================] - 0s 128us/sample - loss: 4.6147 - y1_output_loss: 1.2900 - y2_output_loss: 3.3290 - y1_output_root_mean_squared_error: 1.1413 - y2_output_root_mean_s

Epoch 83/500
614/614 [==============================] - 0s 128us/sample - loss: 4.7483 - y1_output_loss: 1.2708 - y2_output_loss: 3.4386 - y1_output_root_mean_squared_error: 1.1310 - y2_output_root_mean_squared_error: 1.8625 - val_loss: 3.7544 - val_y1_output_loss: 0.8953 - val_y2_output_loss: 2.8009 - val_y1_output_root_mean_squared_error: 0.9443 - val_y2_output_root_mean_squared_error: 1.6919
Epoch 84/500
614/614 [==============================] - 0s 127us/sample - loss: 3.4042 - y1_output_loss: 0.8910 - y2_output_loss: 2.5042 - y1_output_root_mean_squared_error: 0.9451 - y2_output_root_mean_squared_error: 1.5846 - val_loss: 3.2836 - val_y1_output_loss: 0.8350 - val_y2_output_loss: 2.4026 - val_y1_output_root_mean_squared_error: 0.9125 - val_y2_output_root_mean_squared_error: 1.5655
Epoch 85/500
614/614 [==============================] - 0s 128us/sample - loss: 3.6866 - y1_output_loss: 0.9385 - y2_output_loss: 2.7447 - y1_output_root_mean_squared_error: 0.9717 - y2_output_root_mean_s

...

614/614 [==============================] - 0s 127us/sample - loss: 1.2182 - y1_output_loss: 0.2800 - y2_output_loss: 0.9466 - y1_output_root_mean_squared_error: 0.5252 - y2_output_root_mean_squared_error: 0.9708 - val_loss: 2.4138 - val_y1_output_loss: 0.4212 - val_y2_output_loss: 2.0329 - val_y1_output_root_mean_squared_error: 0.6581 - val_y2_output_root_mean_squared_error: 1.4074
Epoch 207/500
614/614 [==============================] - 0s 126us/sample - loss: 1.0594 - y1_output_loss: 0.2032 - y2_output_loss: 0.8518 - y1_output_root_mean_squared_error: 0.4517 - y2_output_root_mean_squared_error: 0.9249 - val_loss: 1.7141 - val_y1_output_loss: 0.1724 - val_y2_output_loss: 1.5259 - val_y1_output_root_mean_squared_error: 0.4131 - val_y2_output_root_mean_squared_error: 1.2424
Epoch 208/500
614/614 [==============================] - 0s 126us/sample - loss: 1.2076 - y1_output_loss: 0.2457 - y2_output_loss: 0.9551 - y1_output_root_mean_squared_error: 0.4962 - y2_output_root_mean_squared_erro

Epoch 227/500
614/614 [==============================] - 0s 126us/sample - loss: 1.0660 - y1_output_loss: 0.2137 - y2_output_loss: 0.8455 - y1_output_root_mean_squared_error: 0.4643 - y2_output_root_mean_squared_error: 0.9222 - val_loss: 1.4924 - val_y1_output_loss: 0.1589 - val_y2_output_loss: 1.3768 - val_y1_output_root_mean_squared_error: 0.4011 - val_y2_output_root_mean_squared_error: 1.1539
Epoch 228/500
614/614 [==============================] - 0s 130us/sample - loss: 1.2788 - y1_output_loss: 0.2497 - y2_output_loss: 1.0319 - y1_output_root_mean_squared_error: 0.4965 - y2_output_root_mean_squared_error: 1.0160 - val_loss: 4.7332 - val_y1_output_loss: 1.0205 - val_y2_output_loss: 3.7723 - val_y1_output_root_mean_squared_error: 0.9817 - val_y2_output_root_mean_squared_error: 1.9415
Epoch 229/500
614/614 [==============================] - 0s 129us/sample - loss: 1.6000 - y1_output_loss: 0.3886 - y2_output_loss: 1.1975 - y1_output_root_mean_squared_error: 0.6257 - y2_output_root_mea

614/614 [==============================] - 0s 126us/sample - loss: 1.0846 - y1_output_loss: 0.2143 - y2_output_loss: 0.8655 - y1_output_root_mean_squared_error: 0.4640 - y2_output_root_mean_squared_error: 0.9324 - val_loss: 1.4010 - val_y1_output_loss: 0.3308 - val_y2_output_loss: 1.1165 - val_y1_output_root_mean_squared_error: 0.5692 - val_y2_output_root_mean_squared_error: 1.0378
Epoch 248/500
614/614 [==============================] - 0s 129us/sample - loss: 1.1302 - y1_output_loss: 0.2397 - y2_output_loss: 0.8900 - y1_output_root_mean_squared_error: 0.4901 - y2_output_root_mean_squared_error: 0.9434 - val_loss: 1.4669 - val_y1_output_loss: 0.1776 - val_y2_output_loss: 1.2810 - val_y1_output_root_mean_squared_error: 0.4234 - val_y2_output_root_mean_squared_error: 1.1348
Epoch 249/500
614/614 [==============================] - 0s 137us/sample - loss: 1.1125 - y1_output_loss: 0.2445 - y2_output_loss: 0.8617 - y1_output_root_mean_squared_error: 0.4950 - y2_output_root_mean_squared_erro

Epoch 268/500
614/614 [==============================] - 0s 126us/sample - loss: 0.8782 - y1_output_loss: 0.1848 - y2_output_loss: 0.6862 - y1_output_root_mean_squared_error: 0.4317 - y2_output_root_mean_squared_error: 0.8318 - val_loss: 1.3876 - val_y1_output_loss: 0.1641 - val_y2_output_loss: 1.2042 - val_y1_output_root_mean_squared_error: 0.4032 - val_y2_output_root_mean_squared_error: 1.1068
Epoch 269/500
614/614 [==============================] - 0s 126us/sample - loss: 0.9099 - y1_output_loss: 0.1892 - y2_output_loss: 0.7152 - y1_output_root_mean_squared_error: 0.4359 - y2_output_root_mean_squared_error: 0.8485 - val_loss: 1.4851 - val_y1_output_loss: 0.2004 - val_y2_output_loss: 1.3128 - val_y1_output_root_mean_squared_error: 0.4526 - val_y2_output_root_mean_squared_error: 1.1315
Epoch 270/500
614/614 [==============================] - 0s 124us/sample - loss: 0.8295 - y1_output_loss: 0.1750 - y2_output_loss: 0.6619 - y1_output_root_mean_squared_error: 0.4199 - y2_output_root_mea

614/614 [==============================] - 0s 130us/sample - loss: 1.7759 - y1_output_loss: 0.4217 - y2_output_loss: 1.3402 - y1_output_root_mean_squared_error: 0.6512 - y2_output_root_mean_squared_error: 1.1627 - val_loss: 1.4081 - val_y1_output_loss: 0.1802 - val_y2_output_loss: 1.2652 - val_y1_output_root_mean_squared_error: 0.4294 - val_y2_output_root_mean_squared_error: 1.1062
Epoch 289/500
614/614 [==============================] - 0s 133us/sample - loss: 0.8454 - y1_output_loss: 0.1752 - y2_output_loss: 0.6672 - y1_output_root_mean_squared_error: 0.4165 - y2_output_root_mean_squared_error: 0.8197 - val_loss: 1.5214 - val_y1_output_loss: 0.3023 - val_y2_output_loss: 1.2327 - val_y1_output_root_mean_squared_error: 0.5581 - val_y2_output_root_mean_squared_error: 1.1000
Epoch 290/500
614/614 [==============================] - 0s 130us/sample - loss: 0.7726 - y1_output_loss: 0.1877 - y2_output_loss: 0.5905 - y1_output_root_mean_squared_error: 0.4273 - y2_output_root_mean_squared_erro

## Evaluate the Model and Plot Metrics

In [ ]:
# Test the model and print loss and mse for both outputs
loss, Y1_loss, Y2_loss, Y1_rmse, Y2_rmse = model.evaluate(x=norm_test_X, y=test_Y)
print("Loss = {}, Y1_loss = {}, Y1_mse = {}, Y2_loss = {}, Y2_mse = {}".format(loss, Y1_loss, Y1_rmse, Y2_loss, Y2_rmse))

In [ ]:
# Plot the loss and mse
Y_pred = model.predict(norm_test_X)
plot_diff(test_Y[0], Y_pred[0], title='Y1')
plot_diff(test_Y[1], Y_pred[1], title='Y2')
plot_metrics(metric_name='y1_output_root_mean_squared_error', title='Y1 RMSE', ylim=6)
plot_metrics(metric_name='y2_output_root_mean_squared_error', title='Y2 RMSE', ylim=7)